In [19]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
import mlflow

In [20]:
train = pd.read_csv('../data/train.csv')

In [21]:
X = train.drop(columns=[
    'ID', 'target1', 'target2', 'target3', 'target4', 'target5',
    'target6', 'target7', 'target8', 'target9', 'target10', 'target11'
])

y = train.drop(columns=[
    'ID', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6',
    'feature7', 'feature8', 'feature9', 'feature10', 'feature11',
    'feature12', 'feature13', 'feature14', 'feature15'
])


In [22]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Hyper-param optmization

In [23]:
import optuna
from lightgbm import LGBMRegressor, callback
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
import mlflow
import numpy as np
import warnings
from sklearn.metrics import root_mean_squared_error

warnings.filterwarnings('ignore')

def objective(trial):
    with mlflow.start_run(run_name=f'Trial-{trial.number}', nested=True):
        # 1) Defino max_depth antes de num_leaves
        max_depth = trial.suggest_int('max_depth', 4, 12)
        num_leaves = trial.suggest_int(
            'num_leaves',
            min(2**(max_depth-1), 31),
            min(2**max_depth, 512)
        )

        params = {
            'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
            'max_depth': max_depth,
            'num_leaves': num_leaves,
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 2.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 2.0, log=True),
            'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.1),
            'random_state': 42,
            'n_jobs': 1,
            'device': 'gpu',      # ou 'cpu' se preferir
            'verbose': -1         # silencia todas as mensagens de treino
        }

        mlflow.log_params(params)

        base = LGBMRegressor(**params)
        model = MultiOutputRegressor(base, n_jobs=1)

        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        rmses = []

        for train_idx, val_idx in kf.split(X):
            X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
            y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

            model.fit(
                X_tr, y_tr,
            )

            y_pred = model.predict(X_va)
            rmses.append(root_mean_squared_error(y_va, y_pred))

        rmse_mean = np.mean(rmses)
        mlflow.log_metric('rmse', rmse_mean)
        return rmse_mean


In [ ]:
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=20,
        n_warmup_steps=10,
        interval_steps=5
    )
)

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('DSWA - Competicao3')
with mlflow.start_run(run_name='LightGBM optimization'):
    study.optimize(objective, n_trials=50)
    
    print("Melhores hiperparâmetros:")
    print(study.best_params)
    print(f"Melhor RMSE: {study.best_value:.4f}")

    # Loga os melhores parâmetros e métrica no MLflow
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_rmse", study.best_value)


[I 2025-07-28 13:45:36,804] A new study created in memory with name: no-name-0d98d14e-2aff-4fc6-aa9e-a560498bbd93
[I 2025-07-28 13:47:31,548] Trial 0 finished with value: 0.5190559942493607 and parameters: {'max_depth': 7, 'num_leaves': 124, 'n_estimators': 3794, 'learning_rate': 0.030405325392865647, 'min_child_samples': 12, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'reg_alpha': 0.531434303862503, 'reg_lambda': 0.038495830758681175, 'min_split_gain': 0.07080725777960455}. Best is trial 0 with value: 0.5190559942493607.


🏃 View run Trial-0 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/5ee7818c3782416a865b5f189be14311
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-28 13:53:28,768] Trial 1 finished with value: 0.5104698205733434 and parameters: {'max_depth': 4, 'num_leaves': 16, 'n_estimators': 4246, 'learning_rate': 0.0033572967053517922, 'min_child_samples': 13, 'subsample': 0.5917022549267169, 'colsample_bytree': 0.6521211214797689, 'reg_alpha': 0.0180714563413956, 'reg_lambda': 0.007207895500630222, 'min_split_gain': 0.029122914019804193}. Best is trial 1 with value: 0.5104698205733434.


🏃 View run Trial-1 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/0dcab82499864bf89e5c86898d437668
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-28 14:00:32,905] Trial 2 finished with value: 0.5222964105364192 and parameters: {'max_depth': 9, 'num_leaves': 98, 'n_estimators': 1814, 'learning_rate': 0.008082071885709252, 'min_child_samples': 25, 'subsample': 0.8925879806965068, 'colsample_bytree': 0.5998368910791798, 'reg_alpha': 0.016283124535115105, 'reg_lambda': 0.035317733939252895, 'min_split_gain': 0.0046450412719997725}. Best is trial 1 with value: 0.5104698205733434.


...

🏃 View run Trial-34 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/803caaecd4db44929f02f67a03aba184
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-28 15:41:36,262] Trial 35 finished with value: 0.5189284337034212 and parameters: {'max_depth': 5, 'num_leaves': 21, 'n_estimators': 3890, 'learning_rate': 0.04586672112975937, 'min_child_samples': 12, 'subsample': 0.8333337163096792, 'colsample_bytree': 0.9533019022223166, 'reg_alpha': 0.0011195753889514737, 'reg_lambda': 0.034389580731365284, 'min_split_gain': 0.06518074134368941}. Best is trial 11 with value: 0.46871354376919394.


🏃 View run Trial-35 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/20f687c79fd64ecd8b8673ba41b24e09
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-28 15:43:49,037] Trial 36 finished with value: 0.46988153106471275 and parameters: {'max_depth': 9, 'num_leaves': 124, 'n_estimators': 4537, 'learning_rate': 0.07236037049257664, 'min_child_samples': 5, 'subsample': 0.8711640681706166, 'colsample_bytree': 0.7448784175434054, 'reg_alpha': 0.00033768218408293364, 'reg_lambda': 0.0036006377726825333, 'min_split_gain': 0.0500006767809659}. Best is trial 11 with value: 0.46871354376919394.


🏃 View run Trial-36 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/076dbcecbda84641af6dbd625ed02e72
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
